In [22]:
from sqlalchemy import create_engine
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv(verbose=True,
            dotenv_path='./.env')
engine = create_engine(f'postgresql://{os.getenv("USERNAME")}:{os.getenv("PASSWORD")}@{os.getenv("HOST")}/{os.getenv("DATABASE")}', 
                       connect_args={'options': '-csearch_path={}'.format('de')})

In [ ]:
%%time
drug_exposure = pd.read_sql_table('drug_exposure', engine)
concept = pd.read_sql_table('concept', engine)
# takes about 3 min

# No.4
drug_exposure 테이블은 환자가 병원에서 처방받은 약의 종류와 처방시작일과 종료일에 대한 정보를 포함하고 있습니다.  
drug_exposure 테이블로부터 선택된 15가지의 약 번호와 약품명이 저장된 첫번째 drugs 테이블이 있으며,  
15가지 약별로 drug_exposure에 저장된 처방건수가 저장된 두번째 prescription_count 테이블이 있습니다.  
마지막으로 drugs 테이블에 해당되는 15가지 약별로 가장 많이 처방되는 약을 짝지어 놓은 drug_pair 테이블이 있습니다.  
3개의 테이블을 사용하여 짝지어진 두번째 약의 처방 건수가 첫번째 약의 처방 건수보다 더 많은 첫번째 약의 약품명을 처방건수 순으로 출력합니다.
- drugs : drug_concept_id(첫번째약 번호), concept_name(약품명)
- prescription_count : drug_concept_id(첫번째약 번호), cnt(처방건수)
- drug_pair : drug_concept_id1(첫번째약 번호), drug_concept_id2(두번째약 번호)


In [7]:
target = [40213154,19078106,19009384,40224172,19127663,1511248,40169216,1539463,19126352,1539411,1332419,40163924,19030765,19106768,19075601]

In [8]:
drugs_list = drug_exposure[['drug_concept_id']].merge(concept[['concept_id', 'concept_name']], 'inner', left_on='drug_concept_id', right_on='concept_id')
drugs_list = drugs_list[drugs_list['drug_concept_id'].isin(target)].drop('concept_id', axis=1)

In [9]:
drugs_group = drugs_list.groupby(['drug_concept_id', 'concept_name'], as_index=False)
drugs_list = drugs_group.size().sort_values(by='size', ascending=False)

In [10]:
drugs_list.head()

,drug_concept_id,concept_name,size
13,40213154,"Influenza, seasonal, injectable, preservative ...",8015
7,19078106,hydrochlorothiazide 25 MG Oral Tablet,3669
4,19009384,"insulin isophane, human 70 UNT/ML / insulin, r...",2825
14,40224172,amlodipine 5 MG / hydrochlorothiazide 12.5 MG ...,2814
10,19127663,atenolol 50 MG / chlorthalidone 25 MG Oral Tab...,2729


### 4-a drugs

In [11]:
drugs = drugs_list[['drug_concept_id', 'concept_name']]
drugs.head()

,drug_concept_id,concept_name
13,40213154,"Influenza, seasonal, injectable, preservative ..."
7,19078106,hydrochlorothiazide 25 MG Oral Tablet
4,19009384,"insulin isophane, human 70 UNT/ML / insulin, r..."
14,40224172,amlodipine 5 MG / hydrochlorothiazide 12.5 MG ...
10,19127663,atenolol 50 MG / chlorthalidone 25 MG Oral Tab...


### 4-b prescription_count

In [12]:
prescription_count = drugs_list[['drug_concept_id', 'size']]
prescription_count.head()

,drug_concept_id,size
13,40213154,8015
7,19078106,3669
4,19009384,2825
14,40224172,2814
10,19127663,2729


### 4-c drug_pair

In [40]:
# pivot table을 통해 인덱스에 날짜, columns에 약품id가 오도록 한다
x =  pd.pivot_table(drug_exposure[['drug_exposure_start_date', 'person_id', 'drug_concept_id', 'visit_occurrence_id']], index=['drug_exposure_start_date', 'person_id'], columns='drug_concept_id').stack(level=0)

In [41]:
x = x.reset_index()
x = x.set_index(['drug_exposure_start_date', 'person_id']).drop(['level_2'], axis=1)

In [44]:
# dot product를 통해 row에서 True를 갖는 값만 열 이름을 값으로 가지도록 dataframe 만듬
x.columns = map(str, x.columns)
x = pd.DataFrame(x.notna().dot(x.columns+(",")).str.rstrip(','), columns=['prescription'])

In [49]:
x['first'] = x.prescription.str.split(",").str[0]
x['second'] = x.prescription.str.split(",").str[1]

In [53]:
x = x[['first', 'second']].reset_index(drop=True)

In [117]:
# 두 쌍인 경우만 남김
pair = x.dropna(axis=0)

In [118]:
# 처방 된 약이 총 몇번이나 처방됐는지 --> rank
tmp = pair.melt()
pair_counts = pd.crosstab(index=tmp['value'], columns=tmp['variable'], margins=True)

In [119]:
rank = pair_counts['All'].sort_values(ascending=False)

In [120]:
drug_pair = pd.DataFrame()

In [121]:
pairs = pair.drop_duplicates()
count = []

In [134]:
# 약품 쌍에서 두 번째 열의 약품이 첫 번째 보다 더 많이 처방됐으면 drug_pairs에 append
for _, row in pairs.iterrows():
    if rank[row['first']] < rank[row['second']]:
        drug_pair = drug_pair.append(row)
        count.append(rank[row['first']])

In [135]:
drug_pair['count'] = count

In [136]:
# count는 첫 번째 열의 약품의 처방된 횟수
# count를 기준으로 drug_pair를 정렬
drug_pair = drug_pair.sort_values(by='count', ascending=False)
drug_pair = drug_pair[['first', 'second']]
drug_pair

,first,second
14814,19009384,40213154
4825,40169281,40213154
20580,1127433,40213154
723,19078106,40213154
4824,19106768,40163924
...,...,...
6209,1150771,1594382
11244,19074841,19078461
17740,19077572,40169281
25521,1114379,1511248
